In [ ]:
print("importing...")
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget


import os
print("importing finished")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print("loading model...")
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))
print("Model loaded successfully")

In [ ]:
print("Running model...")
batch_size = 4
guidance_scale = 15.0
prompt = "a shark"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)
print("Model ran successfully")

In [ ]:
print("Rendering meshes...")
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))
print("Meshes rendered successfully")

In [ ]:
print("Saving meshes to file")
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

# Create the "Meshfiles" directory if it doesn't exist
output_folder = 'Meshfiles'
os.makedirs(output_folder, exist_ok=True)

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()

    # Specify the file paths within the 'output' folder
    ply_file_path = os.path.join(output_folder, f'example_mesh_{i}.ply')
    obj_file_path = os.path.join(output_folder, f'example_mesh_{i}.obj')


    with open(ply_file_path, 'wb') as f:
        t.write_ply(f)
    with open(obj_file_path, 'w') as f:
        t.write_obj(f)

print("Meshes saved")